### Import and data loading

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics.cluster import contingency_matrix
import numpy as np

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import DBSCAN


from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
from sklearn.model_selection import cross_val_predict, train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score




In [2]:
#Import data
kilothonGames = pd.read_csv('KilothonGames.csv')
concepts = pd.read_csv('concepts.csv')
gamerules = pd.read_csv('gamerulesets.csv')
games = pd.read_csv('games.csv')
rulesetconcept = pd.read_csv('rulesetconcepts.csv')
rsc_uct = pd.read_csv('rulesetconceptsuct.csv')
rsc_ab = pd.read_csv('rulesetconceptsab.csv')

### Get infos of the different database

In [3]:
#Shapes

print(concepts.shape)
print(gamerules.shape)
print(games.shape)
print(rulesetconcept.shape)
print(rsc_uct.shape)
print(rsc_ab.shape)

(779, 9)
(1854, 20)
(1458, 26)
(1106316, 5)
(1164417, 5)
(1164417, 5)


In [4]:
# Column names for each df
print(concepts.columns)
print(gamerules.columns)
print(games.columns)
print(rulesetconcept.columns)
print(rsc_uct.columns)
print(rsc_ab.columns)

Index(['Unnamed: 0', 'Id', 'Name', 'Description', 'TypeId', 'DataTypeId',
       'ComputationTypeId', 'TaxonomyString', 'LeafNode'],
      dtype='object')
Index(['Unnamed: 0', 'Id', 'GameId', 'Name', 'NativeName', 'Summary', 'Type',
       'Description', 'Rules', 'Reference', 'Origin', 'Author', 'Publisher',
       'Date', 'OriginPoint', 'EvidenceRange', 'SelfContained', 'Notes',
       'DisableWebApp', 'WishlistRuleset'],
      dtype='object')
Index(['Unnamed: 0', 'Id', 'Name', 'NativeName', 'Description', 'MainRuleset',
       'LudiiRuleset', 'Reference', 'Origin', 'DLPGame', 'PublicGame',
       'knownAliases', 'Author', 'Publisher', 'ProprietaryGame', 'Credit',
       'SeeAlso', 'BGGId', 'OriginPoint', 'EvidenceRange', 'WishlistGame',
       'Notes', 'HelpUs', 'ForceRulesetInLud', 'DisableWebApp',
       'ReconstructionDescriptionDone'],
      dtype='object')
Index(['Unnamed: 0', 'Id', 'RulesetId', 'ConceptId', 'Value'], dtype='object')
Index(['Unnamed: 0', 'Id', 'RulesetId', 'Conc

As discussed, we will only keep the following columns for our final datframe:

- Concepts: ID, LeafNode
- Gamerules: ID, GameID, Name, Type
- Games: ID, Name, MainRuleSet, LudiiRuleset
- Rulesetconcept: ID, RuleSetId, ConceptID, value 

In [5]:
concepts = concepts[~concepts['TaxonomyString'].str.startswith('6')]
concepts = concepts[concepts['LeafNode']!=0]

In [6]:
concepts = concepts.drop([ 'Unnamed: 0', 'Description', 'TypeId', 'DataTypeId', 'ComputationTypeId', 'TaxonomyString'], axis = 1);
gamerules = gamerules.drop(['Unnamed: 0', 'NativeName', 'Summary', 'Description', 'Rules', 'Reference', 'Origin', 'Author', 'Publisher', 'Date', 'OriginPoint', 'EvidenceRange', 'SelfContained', 'Notes', 'DisableWebApp', 'WishlistRuleset', 'Type'], axis = 1);
games = games.drop(['Unnamed: 0','NativeName', 'Description', 'Reference', 'Origin', 'DLPGame', 'PublicGame', 'knownAliases', 'Author', 'Publisher', 'ProprietaryGame', 'Credit', 'SeeAlso', 'BGGId', 'OriginPoint', 'EvidenceRange', 'WishlistGame', 'Notes', 'HelpUs', 'ForceRulesetInLud', 'DisableWebApp', 'ReconstructionDescriptionDone'], axis = 1);
rulesetconcept = rulesetconcept.drop(['Unnamed: 0'], axis = 1)
rsc_uct = rsc_uct.drop(['Unnamed: 0'], axis = 1)
rsc_ab = rsc_ab.drop(['Unnamed: 0'], axis = 1)
concepts

,Id,Name,LeafNode
3,4,Discrete,1
4,5,Realtime,1
6,7,Alternating,1
7,8,Simultaneous,1
8,9,Stochastic,1
...,...,...,...
774,775,DoLudeme,1
775,776,Trigger,1
776,777,PlayoutsPerSecond,1
777,778,MovesPerSecond,1


Now that the df are filtered with the only attributes that we need, we will merge them in a final dataframe.

We will join using the following idea: 

Join Game Ruleset and Games on GameId and Id  --> Merge_One

Join Game Ruleset and Ruleset Concepts on Id and RulesetId --> Merge_two		

Join Concepts and Ruleset Concepts on Id and ConceptId --> Merge_three

In [7]:
def merge(one, two, on, left, right):
  new = pd.merge(one, two, on = on, left_on = left, right_on = right)

  return new

In [8]:
merge_one = merge(games, gamerules, None, 'Id', 'GameId')
merge_two = merge(rulesetconcept, gamerules, None, 'RulesetId', 'Id')
merge_three = merge(concepts, rulesetconcept, None, 'Id', 'ConceptId')

merge_three.drop(['Id_y'], axis=1, inplace=True)
merge_three = merge_three[merge_three['LeafNode']!=0]

Create a dataframe by merging the first three created above

In [9]:
df = merge(merge_one, merge_two, 'GameId', None, None)
df = df.drop([ 'Id_y_y', 'Id_x_x', 'Value', 'Id_x_y', 'ConceptId', 'Name_y', 'Id_y_x'], axis = 1);
df.head()

,Name_x,MainRuleset,LudiiRuleset,GameId,RulesetId,Name
0,Senet,528,529.0,3,529,Bell
1,Senet,528,529.0,3,529,Bell
2,Senet,528,529.0,3,529,Bell
3,Senet,528,529.0,3,529,Bell
4,Senet,528,529.0,3,529,Bell


Do the same merging as merge 3 but for the datasets of UCT and AB

In [10]:
merge_three_uct = pd.merge(concepts, rsc_uct, left_on='Id', right_on='ConceptId')

merge_three_uct.drop(['Id_y'], axis=1, inplace=True)
merge_three_uct = merge_three_uct[merge_three_uct['LeafNode']!=0]
merge_three_uct.head()


,Id_x,Name,LeafNode,RulesetId,ConceptId,Value
0,4,Discrete,1,1656,4,1.0
1,4,Discrete,1,1979,4,1.0
2,4,Discrete,1,35,4,1.0
3,4,Discrete,1,1391,4,1.0
4,4,Discrete,1,1796,4,1.0


In [11]:
merge_three_ab = pd.merge(concepts, rsc_ab, left_on='Id', right_on='ConceptId')

merge_three_ab.drop(['Id_y'], axis=1, inplace=True)
merge_three_ab = merge_three_ab[merge_three_ab['LeafNode']!=0]
merge_three_ab.head()


,Id_x,Name,LeafNode,RulesetId,ConceptId,Value
0,4,Discrete,1,1656,4,1.0
1,4,Discrete,1,1979,4,1.0
2,4,Discrete,1,35,4,1.0
3,4,Discrete,1,1391,4,1.0
4,4,Discrete,1,1796,4,1.0


In [12]:
def new_merge(merge_three):
  one = merge_three.groupby('RulesetId')['Name'].apply(list)
  one = one.to_frame()
  two = pd.merge(one, merge_three, on = 'RulesetId')

  three = two.groupby('RulesetId')['Value'].apply(list)
  three = three.to_frame()

  final = pd.merge(two, three, on = 'RulesetId')

  final.drop(['Name_y', 'LeafNode', 'ConceptId', 'Value_x', 'Id_x'], axis=1, inplace = True)
  final.drop_duplicates(subset = ['RulesetId'], inplace = True)
  return final

In [13]:
new_merge_ = new_merge(merge_three)

In [14]:
new_merge_uct = new_merge(merge_three_uct)

In [15]:
new_merge_ab = new_merge(merge_three_ab)

In [16]:
df = pd.merge(df, new_merge_, on = 'RulesetId')
df.rename(columns = {'Name_x_x' : 'Game', 'Name_x_y' : 'Concepts', 'Value_y' : 'Concepts_values'}, inplace = True)
df = pd.merge(df, new_merge_ab, on='RulesetId')
df.rename(columns = {'Name_x' : 'Concepts_AB', 'Value_y' : 'Concepts_values_AB'}, inplace = True)
df = pd.merge(df, new_merge_uct, on='RulesetId')
df.drop_duplicates(subset = ['RulesetId'], inplace = True)
df.rename(columns = {'Name_x' : 'Concepts_UCT', 'Value_y' : 'Concepts_values_UCT'}, inplace = True)
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Concepts,Concepts_values,Concepts_AB,Concepts_values_AB,Concepts_UCT,Concepts_values_UCT
0,Senet,528,529.0,3,529,Bell,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
21784,Senet,528,529.0,3,939,Jequier,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
43568,Senet,528,529.0,3,922,Kendall,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
65352,Senet,528,529.0,3,937,Kendall All Options,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
87136,Senet,528,529.0,3,923,Kendall Five Pieces,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


In [17]:
c_ab = df.iloc[0]['Concepts_AB']
for i in range(len(c_ab)):
  c_ab[i] = c_ab[i] + '_AB'

c_uct = df.iloc[0]['Concepts_UCT']
for i in range(len(c_uct)):
  c_uct[i] = c_uct[i] + '_UCT'

In [18]:
df[c_ab] = pd.DataFrame(df.Concepts_values_AB.tolist(), index= df.index)

df.drop(['Concepts_AB', 'Concepts_values_AB'], axis=1, inplace = True)
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Concepts,Concepts_values,Concepts_UCT,Concepts_values_UCT,...,SetHiddenWhat_AB,SetHiddenWho_AB,CopyContext_AB,Then_AB,ForEachPiece_AB,DoLudeme_AB,Trigger_AB,PlayoutsPerSecond_AB,MovesPerSecond_AB,OutcomeUniformity_AB
0,Senet,528,529.0,3,529,Bell,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete_UCT, Realtime_UCT, Alternating_UCT, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1358.93,166173.49,NaN
21784,Senet,528,529.0,3,939,Jequier,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,155.01,107657.20,NaN
43568,Senet,528,529.0,3,922,Kendall,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,318.51,81394.19,NaN
65352,Senet,528,529.0,3,937,Kendall All Options,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,321.79,81613.66,NaN
87136,Senet,528,529.0,3,923,Kendall Five Pieces,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,445.57,84795.32,NaN


In [19]:
df[c_uct] = pd.DataFrame(df.Concepts_values_UCT.tolist(), index= df.index)

df.drop(['Concepts_UCT', 'Concepts_values_UCT'], axis=1, inplace = True)
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Concepts,Concepts_values,Discrete_AB,Realtime_AB,...,SetHiddenWhat_UCT,SetHiddenWho_UCT,CopyContext_UCT,Then_UCT,ForEachPiece_UCT,DoLudeme_UCT,Trigger_UCT,PlayoutsPerSecond_UCT,MovesPerSecond_UCT,OutcomeUniformity_UCT
0,Senet,528,529.0,3,529,Bell,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1246.27,152434.05,NaN
21784,Senet,528,529.0,3,939,Jequier,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,126.63,90267.74,NaN
43568,Senet,528,529.0,3,922,Kendall,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,259.06,65906.43,NaN
65352,Senet,528,529.0,3,937,Kendall All Options,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,273.09,66913.65,NaN
87136,Senet,528,529.0,3,923,Kendall Five Pieces,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,396.11,75353.50,NaN


In [20]:
c=df['Concepts'].explode().unique()
df[c] = pd.DataFrame(df.Concepts_values.tolist(), index= df.index)

df.drop(['Concepts', 'Concepts_values'], axis=1, inplace = True)
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Discrete_AB,Realtime_AB,Alternating_AB,Simultaneous_AB,...,SetHiddenValue,SetHiddenWhat,SetHiddenWho,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond
0,Senet,528,529.0,3,529,Bell,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1266.24,154613.78
21784,Senet,528,529.0,3,939,Jequier,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,337.43,87643.94
43568,Senet,528,529.0,3,922,Kendall,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,292.51,72076.99
65352,Senet,528,529.0,3,937,Kendall All Options,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,343.81,74169.23
87136,Senet,528,529.0,3,923,Kendall Five Pieces,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,445.21,84469.25


In [21]:
kilothonGames['RulesetId'] = kilothonGames['RulesetId'].astype('int')
df = pd.merge(df, kilothonGames, on = 'RulesetId')

In [22]:
df.shape

(1104, 1521)

In [23]:
df = df.T.drop_duplicates().T
# df = df.drop_duplicates()

In [24]:
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Discrete_AB,Realtime_AB,Stochastic_AB,AsymmetricPiecesType_AB,...,Union,Intersection,Complement,ConditionalStatement,ControlFlowStatement,CopyContext,DoLudeme,PlayoutsPerSecond,MovesPerSecond,Name
0,Senet,528,529,3,922,Kendall,1,0,1,0,...,1,1,1,1,1,1,1,292.51,72077,Senet
1,Ludus Latrunculorum,32,534,4,534,Schadler 1994,1,0,0,0,...,1,0,0,1,1,0,0,181.25,45676,Ludus Latrunculorum
2,XII Scripta,533,33,5,33,Murray,1,0,1,0,...,1,0,0,1,1,1,1,201.52,51387.4,XII Scripta
3,Mehen,34,542,6,542,Kendall,1,0,1,0,...,1,0,0,1,1,1,1,413.62,105886,Mehen
4,20 Squares,530,35,7,35,Seleucid,1,0,1,0,...,1,0,0,1,1,1,1,1050.14,135749,20 Squares


In [25]:
df.shape

(1104, 669)

In [26]:
df.to_csv('data.csv')